In [13]:
import fitz  # PyMuPDF
import pandas as pd
import re
import os

# Path to the PDF file
pdf_path = r"C:\Users\clint\Desktop\RER\data\Remittance_4\Africa.pdf"

# Check if file exists
if os.path.exists(pdf_path):
    print(f"PDF found: {pdf_path}")
else:
    print(f"PDF not found at: {pdf_path}")
    print("Please check the file path.")

PDF found: C:\Users\clint\Desktop\RER\data\Remittance_3\Remitscope_Africa_central_bank.pdf


In [ ]:
import camelot
import pandas as pd
import re

# Try to extract tables using Camelot
try:
    # Extract tables from first page only
    tables = camelot.read_pdf(pdf_path, pages='1', flavor='stream')
    print(f"Number of tables found: {len(tables)}")
    
    if len(tables) > 0:
        for i, table in enumerate(tables):
            print(f"\nTable {i+1}:")
            print(f"Shape: {table.df.shape}")
            print(table.df.head(10))
            print("="*50)
    else:
        print("No tables found with Camelot")
        
except Exception as e:
    print(f"Error with Camelot: {e}")
    print("Let's try manual parsing...")

Number of tables found: 2

Table 1:
Shape: (90, 6)
                                                   0                    1  \
0  Indicator:\tB\t-\tCentral\tBank\tremittance\ti...                        
1                                   Sending\tcountry  Receive\tcou.. Date   
2                                            Algeria              Senegal   
3                                          Australia             Ethiopia   
4                                                                   Kenya   
5                                                                  Uganda   
6                                            Austria                Kenya   
7                                            Bahamas                Kenya   
8                                            Bahrain                Kenya   
9                                                                 Morocco   

      2                  3              4                             5  
0                          

In [ ]:
# Test both lattice and stream approaches
import os

# Create output directory if it doesn't exist
output_dir = r"C:\Users\clint\Desktop\RER\_output\data\Remittance_3"
os.makedirs(output_dir, exist_ok=True)

print("Testing both Camelot approaches: lattice and stream")
print("="*60)

# Approach 1: Stream flavor (better for tables without clear borders)
print("\n1. TESTING STREAM FLAVOR")
print("-" * 30)
try:
    tables_stream = camelot.read_pdf(pdf_path, pages='1', flavor='stream')
    print(f"Stream - Number of tables found: {len(tables_stream)}")
    
    if len(tables_stream) > 0:
        for i, table in enumerate(tables_stream):
            print(f"\nStream Table {i+1}:")
            print(f"Shape: {table.df.shape}")
            print(f"Accuracy: {table.accuracy:.2f}")
            print("Preview (first 5 rows):")
            print(table.df.head())
            
            # Save to CSV
            csv_filename = f"stream_table_{i+1}.csv"
            csv_path = os.path.join(output_dir, csv_filename)
            table.df.to_csv(csv_path, index=False)
            print(f"Saved to: {csv_path}")
            print("-" * 40)
    else:
        print("No tables found with stream flavor")
        
except Exception as e:
    print(f"Error with stream flavor: {e}")

print("\n" + "="*60)

# Approach 2: Lattice flavor (better for tables with clear borders)
print("\n2. TESTING LATTICE FLAVOR")
print("-" * 30)
try:
    tables_lattice = camelot.read_pdf(pdf_path, pages='1', flavor='lattice')
    print(f"Lattice - Number of tables found: {len(tables_lattice)}")
    
    if len(tables_lattice) > 0:
        for i, table in enumerate(tables_lattice):
            print(f"\nLattice Table {i+1}:")
            print(f"Shape: {table.df.shape}")
            print(f"Accuracy: {table.accuracy:.2f}")
            print("Preview (first 5 rows):")
            print(table.df.head())
            
            # Save to CSV
            csv_filename = f"lattice_table_{i+1}.csv"
            csv_path = os.path.join(output_dir, csv_filename)
            table.df.to_csv(csv_path, index=False)
            print(f"Saved to: {csv_path}")
            print("-" * 40)
    else:
        print("No tables found with lattice flavor")
        
except Exception as e:
    print(f"Error with lattice flavor: {e}")

print("\n" + "="*60)
print("SUMMARY:")
print(f"Output directory: {output_dir}")
print("Check the CSV files to compare the results from both approaches.")

Testing both Camelot approaches: lattice and stream

1. TESTING STREAM FLAVOR
------------------------------
Stream - Number of tables found: 2

Stream Table 1:
Shape: (90, 6)
Accuracy: 99.82
Preview (first 5 rows):
                                                   0                    1  \
0  Indicator:\tB\t-\tCentral\tBank\tremittance\ti...                        
1                                   Sending\tcountry  Receive\tcou.. Date   
2                                            Algeria              Senegal   
3                                          Australia             Ethiopia   
4                                                                   Kenya   

      2                  3              4                             5  
0                                                                        
1                    Value           Unit                        Source  
2  2021        0.183414825  USD\tmillions                         BCEAO  
3  2020        13.5961751

In [16]:
# Load and clean the extracted data
print("ANALYZING EXTRACTED DATA")
print("="*50)

# Load the CSV files
stream_table1_path = os.path.join(output_dir, "stream_table_1.csv")
stream_table2_path = os.path.join(output_dir, "stream_table_2.csv")

if os.path.exists(stream_table1_path):
    df1 = pd.read_csv(stream_table1_path)
    print("Stream Table 1 - Raw data:")
    print(f"Shape: {df1.shape}")
    print(df1.head(10))
    print("\n" + "-"*30)

if os.path.exists(stream_table2_path):
    df2 = pd.read_csv(stream_table2_path)
    print("Stream Table 2 - Raw data:")
    print(f"Shape: {df2.shape}")
    print(df2.head(10))
    print("\n" + "-"*30)

# Clean and combine the data
print("\nCLEANING THE DATA...")

# Function to clean the dataframes
def clean_remittance_data(df):
    """Clean the extracted remittance data"""
    # Skip the first few rows that contain headers
    clean_df = df.copy()
    
    # Find where the actual data starts (after header rows)
    data_start = 0
    for i, row in clean_df.iterrows():
        if 'Sending' in str(row.iloc[0]) and 'country' in str(row.iloc[0]):
            data_start = i + 1
            break
    
    if data_start > 0:
        clean_df = clean_df.iloc[data_start:].reset_index(drop=True)
    
    # Remove empty rows
    clean_df = clean_df.dropna(how='all').reset_index(drop=True)
    
    return clean_df

# Clean both tables
if 'df1' in locals():
    clean_df1 = clean_remittance_data(df1)
    print(f"Cleaned Table 1 shape: {clean_df1.shape}")
    print("Sample of cleaned data:")
    print(clean_df1.head())
    
    # Save cleaned data
    clean_path1 = os.path.join(output_dir, "stream_table_1_cleaned.csv")
    clean_df1.to_csv(clean_path1, index=False)
    print(f"Cleaned data saved to: {clean_path1}")

if 'df2' in locals():
    clean_df2 = clean_remittance_data(df2)
    print(f"\nCleaned Table 2 shape: {clean_df2.shape}")
    print("Sample of cleaned data:")
    print(clean_df2.head())
    
    # Save cleaned data
    clean_path2 = os.path.join(output_dir, "stream_table_2_cleaned.csv")
    clean_df2.to_csv(clean_path2, index=False)
    print(f"Cleaned data saved to: {clean_path2}")

print(f"\nAll files saved in: {output_dir}")

ANALYZING EXTRACTED DATA
Stream Table 1 - Raw data:
Shape: (90, 6)
                                                   0                    1  \
0  Indicator:\tB\t-\tCentral\tBank\tremittance\ti...                  NaN   
1                                   Sending\tcountry  Receive\tcou.. Date   
2                                            Algeria              Senegal   
3                                          Australia             Ethiopia   
4                                                NaN                Kenya   
5                                                NaN               Uganda   
6                                            Austria                Kenya   
7                                            Bahamas                Kenya   
8                                            Bahrain                Kenya   
9                                                NaN              Morocco   

        2                  3              4                             5  
0     NaN

In [ ]:
# Summary of extraction results
print("EXTRACTION SUMMARY")
print("="*50)
print("✅ Successfully tested both Camelot approaches:")
print("   • Stream flavor: Extracting tables from page 1 only")
print("   • Lattice flavor: Extracting tables from page 1 only")
print()
print("📊 Extracted data from page 1 contains:")
print("   • Remittance data tables from the first page of the PDF")
print()
print("💾 Files created:")
print("   • stream_table_1.csv - Raw extracted table from page 1")
print("   • stream_table_1_cleaned.csv - Cleaned table from page 1")
print("   • Additional tables if found on page 1")
print()
print(f"📁 Location: {output_dir}")
print()
print("🎯 Recommendation: Use STREAM flavor for this PDF type")
print("   Stream flavor works better for PDFs without clear table borders")
print("   Now extracting from page 1 only as requested")

EXTRACTION SUMMARY
✅ Successfully tested both Camelot approaches:
   • Stream flavor: Found 2 tables with high accuracy (99.82% and 99.73%)
   • Lattice flavor: Found 0 tables (PDF likely doesn't have clear borders)

📊 Extracted data contains:
   • Table 1: 90 rows × 6 columns - Main remittance data
   • Table 2: 43 rows × 5 columns - Additional remittance data

💾 Files created:
   • stream_table_1.csv - Raw extracted table 1
   • stream_table_2.csv - Raw extracted table 2
   • stream_table_1_cleaned.csv - Cleaned table 1
   • stream_table_2_cleaned.csv - Cleaned table 2

📁 Location: C:\Users\clint\Desktop\RER\_output\data\Remittance_3

🎯 Recommendation: Use STREAM flavor for this PDF type
   Stream flavor works better for PDFs without clear table borders


In [18]:
# DIFFERENCE BETWEEN CLEANED AND UNCLEANED DATA
print("COMPARISON: UNCLEANED vs CLEANED DATA")
print("="*60)

print("📊 UNCLEANED DATA (Raw from PDF extraction):")
print("-" * 40)
print("Table 1 - First 10 rows of raw data:")
print(df1.head(10))
print(f"Shape: {df1.shape}")
print()

print("Table 2 - First 10 rows of raw data:")
print(df2.head(10))
print(f"Shape: {df2.shape}")
print()

print("🧹 CLEANED DATA (After processing):")
print("-" * 40)
print("Table 1 - First 10 rows of cleaned data:")
print(clean_df1.head(10))
print(f"Shape: {clean_df1.shape}")
print()

print("Table 2 - First 10 rows of cleaned data:")
print(clean_df2.head(10))
print(f"Shape: {clean_df2.shape}")
print()

print("🔍 KEY DIFFERENCES:")
print("-" * 40)
print("1. HEADER ROWS: Uncleaned data contains PDF metadata and headers")
print("   - Raw data includes indicator descriptions, column headers")
print("   - Cleaned data starts from actual data rows")
print()
print("2. EMPTY ROWS: Uncleaned data may have empty/null rows")
print("   - Raw data preserves PDF structure including spaces")
print("   - Cleaned data removes empty rows")
print()
print("3. DATA STRUCTURE: Cleaned data is more analysis-ready")
print("   - Raw data preserves original PDF formatting")
print("   - Cleaned data has consistent column structure")
print()
print("4. SIZE DIFFERENCE:")
print(f"   - Table 1: {df1.shape[0]} rows → {clean_df1.shape[0]} rows")
print(f"   - Table 2: {df2.shape[0]} rows → {clean_df2.shape[0]} rows")

COMPARISON: UNCLEANED vs CLEANED DATA
📊 UNCLEANED DATA (Raw from PDF extraction):
----------------------------------------
Table 1 - First 10 rows of raw data:
                                                   0                    1  \
0  Indicator:\tB\t-\tCentral\tBank\tremittance\ti...                  NaN   
1                                   Sending\tcountry  Receive\tcou.. Date   
2                                            Algeria              Senegal   
3                                          Australia             Ethiopia   
4                                                NaN                Kenya   
5                                                NaN               Uganda   
6                                            Austria                Kenya   
7                                            Bahamas                Kenya   
8                                            Bahrain                Kenya   
9                                                NaN              Moro